In [1]:
import scipy.io as sci
import os

import numpy as np
import pandas as pd

In [2]:
path = '/home/kerrykim/jupyter_notebook/5. eyetracking/mastercode/'

mat = sci.loadmat(path+'metadata.mat') # loadmat을 하는 경우 mat파일을 python dictionary형태로 load
mat

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Aug 10 15:19:56 2017',
 '__version__': '1.0',
 '__globals__': [],
 'frameIndex': array([[   0],
        [   1],
        [   2],
        ...,
        [2988],
        [2989],
        [2990]], dtype=int32),
 'labelDotXCam': array([[ 1.064     ],
        [ 1.064     ],
        [ 1.064     ],
        ...,
        [10.24670196],
        [10.24670196],
        [10.24670196]]),
 'labelDotYCam': array([[-6.0055    ],
        [-6.0055    ],
        [-6.0055    ],
        ...,
        [ 1.97545637],
        [ 1.97545637],
        [ 1.97545637]]),
 'labelFaceGrid': array([[ 6, 10, 13, 13],
        [ 6, 10, 13, 13],
        [ 6, 10, 13, 13],
        ...,
        [ 7,  7,  7,  7],
        [ 7,  7,  7,  7],
        [ 7,  7,  7,  7]], dtype=uint8),
 'labelRecNum': array([[   2],
        [   2],
        [   2],
        ...,
        [3523],
        [3523],
        [3523]], dtype=int16),
 'labelTest': array([[0],
        [0],
      

In [3]:
np.set_printoptions(threshold=10, linewidth=10) #np.array의 ... 조정, np.set_printoptions(threshold=np.inf, linewidth=np.inf)

In [4]:
mat['labelRecNum'].reshape(1490959,)  # 사전형으로 뽑아낸 mat 파일의 value에는 리스트가 두개 씌워져 있어서 하나를 제거

array([   2,
          2,
          2,
       ...,
       3523,
       3523,
       3523],
      dtype=int16)

In [5]:
mask= mat['labelVal']
np.argwhere(mask)[:,0]

array([  20769,
         20770,
         20771,
       ...,
       1476102,
       1476103,
       1476104])

In [6]:
result = pd.DataFrame({'labelRecNum' : mat['labelRecNum'].reshape(1490959,),
                       'frameIndex' : mat['frameIndex'].reshape(1490959,),
                       'labelDotXCam' : mat['labelDotXCam'].reshape(1490959,),
                       'labelDoxYcam' : mat['labelDotYCam'].reshape(1490959,),
                       'labelTrain' : mat['labelTrain'].reshape(1490959,),
                       'labelVal' : mat['labelVal'].reshape(1490959,),
                       'labelTest' : mat['labelTest'].reshape(1490959,)})

# labelFaceGrid = pd.DataFrame(mat['labelFaceGrid'].T)
result

,labelRecNum,frameIndex,labelDotXCam,labelDoxYcam,labelTrain,labelVal,labelTest
0,2,0,1.064000,-6.005500,1,0,0
1,2,1,1.064000,-6.005500,1,0,0
2,2,2,1.064000,-6.005500,1,0,0
3,2,3,1.064000,-6.005500,1,0,0
4,2,4,3.251485,-1.993073,1,0,0
...,...,...,...,...,...,...,...
1490954,3523,2986,10.246702,1.975456,0,0,1
1490955,3523,2987,10.246702,1.975456,0,0,1
1490956,3523,2988,10.246702,1.975456,0,0,1
1490957,3523,2989,10.246702,1.975456,0,0,1


In [7]:
# df1[df1.E == 'train']

result[result.labelVal==1]['labelRecNum'].value_counts().sort_index(ascending=True)

135     1608
213     1252
267     1122
471      510
507     2009
547      472
618      565
746      563
776     1535
803       25
926      713
960     1562
1103    1759
1119    2260
1220    1781
1248     229
1274     236
1297     358
1319    1900
1373    1236
1404    1422
1475    1032
1509    1860
1543     353
1646     700
1760    2321
1773    2405
1786    1717
1845    1446
1998     259
2060     727
2133    2012
2166    1045
2217    1270
2236     281
2265    2404
2416     951
2593     909
2650    1068
2731     698
2736     507
2869    1949
3039    1533
3093    1436
3214    2001
3232     194
3258    1493
3312    2425
3349     666
3450     701
Name: labelRecNum, dtype: int64

In [8]:
result[result.labelTest==1]['labelRecNum'].value_counts().sort_index(ascending=True)

10      1926
110      677
126     1240
178      706
190      260
        ... 
3366     513
3404     734
3412    1364
3451     246
3523    2053
Name: labelRecNum, Length: 150, dtype: int64

In [49]:
mask = [[0],[0],[0],[0],[1]]
np.argwhere(mask)[:, 0]

array([4])

In [50]:
result[result.labelVal==1]

,labelRecNum,frameIndex,labelDotXCam,labelDoxYcam,labelTrain,labelVal,labelTest
20769,135,0,6.172245,-3.432166,0,1,0
20770,135,1,6.172245,-3.432166,0,1,0
20771,135,2,6.172245,-3.432166,0,1,0
20772,135,3,4.758891,-19.151955,0,1,0
20773,135,4,4.758891,-19.151955,0,1,0
...,...,...,...,...,...,...,...
1476100,3450,1026,3.395929,-1.617218,0,1,0
1476101,3450,1027,3.395929,-1.617218,0,1,0
1476102,3450,1028,3.395929,-1.617218,0,1,0
1476103,3450,1029,3.395929,-1.617218,0,1,0


In [9]:
result[result.labelRecNum==6][result.frameIndex==0]

/home/kerrykim/anaconda3/envs/cuda_python_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,labelRecNum,frameIndex,labelDotXCam,labelDoxYcam,labelTrain,labelVal,labelTest
159,6,0,1.93875,-5.5845,1,0,0


In [10]:
result[result.labelRecNum==6][result.frameIndex==1]

/home/kerrykim/anaconda3/envs/cuda_python_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,labelRecNum,frameIndex,labelDotXCam,labelDoxYcam,labelTrain,labelVal,labelTest
160,6,1,1.93875,-5.5845,1,0,0


In [11]:
result[result.labelRecNum==6][result.frameIndex==2]

/home/kerrykim/anaconda3/envs/cuda_python_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,labelRecNum,frameIndex,labelDotXCam,labelDoxYcam,labelTrain,labelVal,labelTest
161,6,2,1.93875,-5.5845,1,0,0


In [12]:
result[result.labelRecNum==6][result.frameIndex==3]

/home/kerrykim/anaconda3/envs/cuda_python_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,labelRecNum,frameIndex,labelDotXCam,labelDoxYcam,labelTrain,labelVal,labelTest
162,6,3,1.93875,-5.5845,1,0,0


In [61]:
mat['labelTrain'][0:2285]=1
mat['labelTrain'][2286:3909]=0

In [62]:
mat['labelVal'][0:2285]=0
mat['labelVal'][2286:3084]=1 #이상하네
mat['labelVal'][3084:3909]=0

In [63]:
mat['labelTest'][0:3083]=0
mat['labelTest'][3084:3909]=1

In [64]:
mat['labelTest'][0:3909].shape

(3909, 1)

In [65]:
result = pd.DataFrame({'labelRecNum' : mat['labelRecNum'][0:3909].reshape(3909,),
                       'frameIndex' : mat['frameIndex'][0:3909].reshape(3909,),
                       'labelDotXCam' : mat['labelDotXCam'][0:3909].reshape(3909,),
                       'labelDoxYcam' : mat['labelDotYCam'][0:3909].reshape(3909,),
                       'labelTrain' : mat['labelTrain'][0:3909].reshape(3909,),
                       'labelVal' : mat['labelVal'][0:3909].reshape(3909,),
                       'labelTest' : mat['labelTest'][0:3909].reshape(3909,)})

result

,labelRecNum,frameIndex,labelDotXCam,labelDoxYcam,labelTrain,labelVal,labelTest
0,2,0,1.064000,-6.005500,1,0,0
1,2,1,1.064000,-6.005500,1,0,0
2,2,2,1.064000,-6.005500,1,0,0
3,2,3,1.064000,-6.005500,1,0,0
4,2,4,3.251485,-1.993073,1,0,0
...,...,...,...,...,...,...,...
3904,33,962,-7.238552,-1.264138,0,0,1
3905,33,963,-7.238552,-1.264138,0,0,1
3906,33,964,-9.082433,0.050276,0,0,1
3907,33,965,-9.082433,0.050276,0,0,1


In [66]:
result.to_csv('./metadata.csv',index=False)
#mat['labelTest'][0:15141]

In [67]:
sci.savemat('./metadata_small.mat', {'labelRecNum' : mat['labelRecNum'][0:3909],
                       'frameIndex' : mat['frameIndex'][0:3909],
                       'labelDotXCam' : mat['labelDotXCam'][0:3909],
                       'labelDotYCam' : mat['labelDotYCam'][0:3909],
                       'labelFaceGrid' : mat['labelFaceGrid'][0:3909],
                       'labelTrain' : mat['labelTrain'][0:3909],
                       'labelVal' : mat['labelVal'][0:3909],
                       'labelTest' : mat['labelTest'][0:3909]} )

In [34]:
mat2 = sci.loadmat('./metadata_small.mat') # loadmat을 하는 경우 mat파일을 python dictionary형태로 load
mat2

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Sun Nov 22 04:55:24 2020',
 '__version__': '1.0',
 '__globals__': [],
 'labelRecNum': array([[  2],
        [  2],
        [  2],
        ...,
        [123],
        [123],
        [123]],
       dtype=int16),
 'frameIndex': array([[   0],
        [   1],
        [   2],
        ...,
        [1358],
        [1359],
        [1360]],
       dtype=int32),
 'labelDotXCam': array([[1.064     ],
        [1.064     ],
        [1.064     ],
        ...,
        [3.64302465],
        [3.64302465],
        [3.64302465]]),
 'labelDotYCam': array([[-6.0055  ],
        [-6.0055  ],
        [-6.0055  ],
        ...,
        [-0.969375],
        [-0.969375],
        [-0.969375]]),
 'labelFaceGrid': array([[ 6,
         10,
         13,
         13],
        [ 6,
         10,
         13,
         13],
        [ 6,
         10,
         13,
         13],
        ...,
        [ 4,
          7,
         12,
         12],
        [ 4,
    

In [14]:
mat['labelFaceGrid'][0:37956].shape

(37956, 4)

In [128]:
result = pd.DataFrame({'labelRecNum' : mat['labelRecNum'][0:37956].reshape(37956,),
                       'frameIndex' : mat['frameIndex'][0:37956].reshape(37956,),
                       'labelDotXCam' : mat['labelDotXCam'][0:37956].reshape(37956,),
                       'labelDoxYcam' : mat['labelDotYCam'][0:37956].reshape(37956,),
                       'labelFaceGrid' : mat['labelFaceGrid'][0:37956].reshape(37956,),
                       'labelTrain' : mat['labelTrain'][0:37956].reshape(37956,),
                       'labelVal' : mat['labelVal'][0:37956].reshape(37956,),
                       'labelTest' : mat['labelTest'][0:37956].reshape(37956,)})

result

,labelRecNum,frameIndex,labelDotXCam,labelDoxYcam,labelTrain,labelVal,labelTest
0,2,0,1.064000,-6.005500,1,0,0
1,2,1,1.064000,-6.005500,1,0,0
2,2,2,1.064000,-6.005500,1,0,0
3,2,3,1.064000,-6.005500,1,0,0
4,2,4,3.251485,-1.993073,1,0,0
...,...,...,...,...,...,...,...
37951,156,1313,3.716244,2.214500,0,0,1
37952,156,1314,3.716244,2.214500,0,0,1
37953,156,1315,3.716244,2.214500,0,0,1
37954,156,1316,1.426988,1.064000,0,0,1


In [4]:
path = '/home/kerrykim/jupyter_notebook/5. eyetracking/mastercode/'
mat3 = sci.loadmat('./mean_face_224.mat') # loadmat을 하는 경우 mat파일을 python dictionary형태로 load
mat3

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Fri Apr 29 04:29:44 2016',
 '__version__': '1.0',
 '__globals__': [],
 'image_mean': array([[[112.809074, 129.45522 , 144.93938 ],
         [111.890465, 128.60057 , 144.06122 ],
         [111.20556 , 127.57935 , 143.36815 ],
         ...,
         [108.1978  , 124.373665, 141.28052 ],
         [109.124214, 125.0978  , 142.07185 ],
         [109.79371 , 125.89104 , 142.74463 ]],
 
        [[112.47665 , 129.04677 , 144.58781 ],
         [111.5318  , 128.22336 , 143.7069  ],
         [110.81582 , 127.210175, 142.99075 ],
         ...,
         [107.88884 , 124.02679 , 140.97693 ],
         [108.785065, 124.7553  , 141.75473 ],
         [109.4146  , 125.57967 , 142.38727 ]],
 
        [[112.102455, 128.64438 , 144.22438 ],
         [111.161934, 127.808365, 143.33029 ],
         [110.4569  , 126.812454, 142.62677 ],
         ...,
         [107.53539 , 123.6653  , 140.62279 ],
         [108.42261 , 124.39224 , 141.3968  ],


In [5]:
mat3['image_mean'].shape

(224, 224, 3)

In [97]:
result[result.labelRecNum==149][result.frameIndex==1376]

/home/kerrykim/anaconda3/envs/cuda_python_3.6/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


,labelRecNum,frameIndex,labelDotXCam,labelDoxYcam,labelTrain,labelVal,labelTest
33245,149,1376,8.833097,-1.061022,0,1,0


In [6]:
'''
- dataset_origin에는 동영상의 원본프레임이 보존되어 있고 dataset에는 얼굴/눈으로 인식되지 않은 frame의 사진은 삭제되어 있다.
- dataset_origin에서 얼굴 확인이 되는 frame 추출 후 그 frame에 대한 Left eye, Right eye가 추출되어 있다. (즉 얼굴/눈의 #frame은 같다.)
- dotInfo.json에 있는 Xpts, Ypts에 대한 정보는 코드에 사용되지 않음
- 총 사진의 갯수 1490959*3개 (왜냐하면 폴더가 3개이기 때문)



* metadata.mat의 정보는 폴더 하나만 고려한 기준이다 (1490959개) *


- labelRecNum : 사진의 갯수만큼 폴더명 인덱스를 부여해놓은 리스트.
                ex> 2, 2, 2 ..., 3, 3, ..., .. 
                    / 2번 폴더에 56개 사진이 있으므로 56개의 2가 존재, 3번 폴더에 86개 사진있으므로 86개만큼 3이 존재 ...
                
                
- frameIndex : 전처리된 dataset의 폴더 안에 있는 frame 번호 
                ex> 0, 1, 2 ... 97, 0, 1, 2, .... 
                    / 2번 폴더안에 97번 까지 프레임번호의 사진이 있고 그다음 3번 폴더의 프레임번호가 시작됨. 

labelDotXCam / labelDotYCam : 정확하게는 잘 모르겠는데 카메라의 중심으로부터 상대적 거리로 보임. 실험자들이 각각 다른 디바이스로 데이터를 만들어서
                                일관적인 값을 가져오기 위해 unified prediction space를 이용. 거리는 센티미터 기준이며 실제로 실험자들은
                                디바이스를 움직이면서 데이터를 만들지만 labelDotX/YCam은 카메라가 고정된 한 포지션이라 가정.
                                Ycam의 경우 핸드폰 스크린위에 카메라가 있기 때문에 negative한 값을 가질 것이며 만약 반대로 디바이스를 든 경우
                                positive한 값을 가질수도 있음. (screen.json 파일에 디바이스 그립에 따른 orientation에 대한 정보가 있다.)

labelFaceGrid : 전처리전 사진에 대한 grid의 scale은 25by25로 지정되어 있다.
                ex> 6, 10, 13, 13인 경우, 좌측 상단 위치 = 6,10 / 가로 : 13 / 세로 : 13의 face grid를 갖는다.
                
labelTrain / labelVal / labelTest : 전체 데이터 사진에 대한 Train/Val/Test Index. 
                                    랜덤샘플링을 해서 1271명은 학습데이터, 50명은 검증데이터, 150명은 테스트 데이터로 사용.
'''

'\n- dataset_origin에는 동영상의 원본프레임이 보존되어 있고 dataset에는 얼굴/눈으로 인식되지 않은 frame의 사진은 삭제되어 있다.\n- dataset_origin에서 얼굴 확인이 되는 frame 추출 후 그 frame에 대한 Left eye, Right eye가 추출되어 있다. (즉 얼굴/눈의 #frame은 같다.)\n- dotInfo.json에 있는 Xpts, Ypts에 대한 정보는 코드에 사용되지 않음\n- 총 사진의 갯수 1490959*3개 (왜냐하면 폴더가 3개이기 때문)\n\n\n\n* metadata.mat의 정보는 폴더 하나만 고려한 기준이다 (1490959개) *\n\n\n- labelRecNum : 사진의 갯수만큼 폴더명 인덱스를 부여해놓은 리스트.\n                ex> 2, 2, 2 ..., 3, 3, ..., .. \n                    / 2번 폴더에 56개 사진이 있으므로 56개의 2가 존재, 3번 폴더에 86개 사진있으므로 86개만큼 3이 존재 ...\n                \n                \n- frameIndex : 전처리된 dataset의 폴더 안에 있는 frame 번호 \n                ex> 0, 1, 2 ... 97, 0, 1, 2, .... \n                    / 2번 폴더안에 97번 까지 프레임번호의 사진이 있고 그다음 3번 폴더의 프레임번호가 시작됨. \n\nlabelDotXCam / labelDotYCam : 정확하게는 잘 모르겠는데 카메라의 중심으로부터 상대적 거리로 보임. 실험자들이 각각 다른 디바이스로 데이터를 만들어서\n                                일관적인 값을 가져오기 위해 unified prediction space를 이용. 거리는 센티미터 기준이며 실제로 실험자들은\n                                디바이스를 움직이면서 데이터를 만들지만 l

In [17]:
mat_2 = sci.loadmat('mean_face_224.mat')
mat_2

{'__header__': b'MATLAB 5.0 MAT-file, Platform: GLNXA64, Created on: Fri Apr 29 04:29:44 2016',
 '__version__': '1.0',
 '__globals__': [],
 'image_mean': array([[[112.809074,
          129.45522 ,
          144.93938 ],
         [111.890465,
          128.60057 ,
          144.06122 ],
         [111.20556 ,
          127.57935 ,
          143.36815 ],
         ...,
         [108.1978  ,
          124.373665,
          141.28052 ],
         [109.124214,
          125.0978  ,
          142.07185 ],
         [109.79371 ,
          125.89104 ,
          142.74463 ]],
 
        [[112.47665 ,
          129.04677 ,
          144.58781 ],
         [111.5318  ,
          128.22336 ,
          143.7069  ],
         [110.81582 ,
          127.210175,
          142.99075 ],
         ...,
         [107.88884 ,
          124.02679 ,
          140.97693 ],
         [108.785065,
          124.7553  ,
          141.75473 ],
         [109.4146  ,
          125.57967 ,
          142.38727 ]],
 
        [

In [ ]:
'''
mean_face_224 / ... /...

이건 이미지를 모두 더해서 평균을 구할 수 없으니까 각 이미지 한장에 대한 평균을 구해서 (한 차원에 3개 데이터가 있으니 RGB에 대한 평균을 빼는듯?)
빼가지고 standardization을 함 (데이터의 평균을 0으로 만듬)
'''